# emTools

### a Python module to interact with SerialEM to enable automated Transmission Electron Microscopy

## Table of contents
1. [Installation instructions](#installation)
<br>
  - How to use this jupyter notebook
  - Install necessary dependencies in Python
  - [install/update emTools](#install1)  
<br/>
2. [Getting started](#howtos)
<br>  
  - [Tutorial 1: Sort a Navigator](#tutorial1)
  - [Turorial 2: Import pixel coordinates](#tutorial2)  
<br/>
3. [Setting up KNIME](#knime)



## Installation instructions<a name="installation"></a>

### Prepare your PC for the use of Python and emTools

- first we will install Python and then use it to fetch the packages needed for emTools to work.
- Install Anaconda from [here](https://www.anaconda.com/download/ "Download Anaconda"). EmTools has been tested to work with both Python 2 and 3. Version 3 is recommended as 2.7 is approaching the end of its support frame. Choose the appropriate OS.
- Start the Anaconda [Navigator](http://docs.anaconda.com/anaconda/user-guide/getting-started/ "Getting started with Anaconda").
- as explained in [section 4](http://docs.anaconda.com/anaconda/user-guide/getting-started/#run-python-in-a-jupyter-notebook "Run Python in a Jupyter Notebook"), install and start a Jupyter notebook.
- Download this [tutorial file](https://git.embl.de/schorb/serialem_tools/raw/master/emTools.ipynb) (use "save link as..."), open it in your Jupyter session and follow the instructions.

----
### How to use this jupyter notebook

In this document there are text blocks and code blocks that can be edited and executed.
In order to run a code block, click into it to activate and then press `CTRL+ Enter`. There is a Python logo at the top right of the notebook that indicates whether there are still operations running.

### Install necessary dependencies in Python

The following code block uses Anaconda to fetch necessary packages. Start the code by pressing `CTRL+ Enter` inside the code block.

In [ ]:
# this will load the package needed for python package management
import conda.cli

# this will install the dependencies
conda.cli.main('conda', 'install',  '-y', 'numpy', 'scipy', 'pandas', 'scikit-image', 'git', 'pip')
conda.cli.main('conda', 'install',  '-y', '-c conda-forge', 'tifffile')

## install/update emTools <a name="install1"></a>

Now we can install emTools using pip, another package manager for Python. Start by pressing `CTRL+ Enter` inside the code block. 

To update to the current version, simply execut this code block anytime.

In [8]:
# this will load the pip package needed for python package management
from pip._internal import main as pipmain

# this will install the current version of emTools
pipmain(['install','git+https://git.embl.de/schorb/serialem_tools'])

  Cloning https://git.embl.de/schorb/serialem_tools to /tmp/pip-req-build-40ArGs
  Running setup.py bdist_wheel for emtools: started
  Running setup.py bdist_wheel for emtools: finished with status 'done'
  Stored in directory: /tmp/pip-ephem-wheel-cache-tiT1t2/wheels/df/10/5b/df26f629698cb6cf14998c6211bee9dda9e936fca42d632436
Successfully built emtools


You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


0

----



## Getting started <a name="howtos"></a>



### Tutorial 1: Sort a navigator file<a name="tutorial1"></a>

Have a look at the files in the directory ["`tutorials/Tutorial1-sorting`"](https://git.embl.de/schorb/serialem_tools/tree/master/tutorials/Tutorial1-sorting). If you open the navigator file `nav1.nav` in SerialEM, you will get the following:

#### the Navigator
![The navigator](images/nav1_1.png)

If you double-click the `Map` item, you will see this:
#### the Map
![The cells](images/nav1.png)

We have a series of points for each cell. However they are not listed such that all points lying within one cell are acquired in a row.

Let's use emTools to create a sorted Navigator file.


#### Reading in the Navigator

First, we will import the emTools module and thus makei its functions available for the Python script.
Then we will read in the Navigator file.



In [ ]:
# import the emTools module and make its functions available
import emtools as em

# load the navigator file
navfile = 'tutorials/Tutorial1-sorting/nav1.nav'
navlines = em.loadtext(navfile)

# now we have a list of strings for each row of the file, let's look at the first 10.
print('first 10 lines of the navigator: \n')
print(navlines[0:10])
print('\n----\n')


# let's convert them into a list of dictionaries with one entry for each navigator Item
allitems = em.fullnav(navlines)
# Let look at one item
print('Item number 7 from the navigator file: \n')
print(allitems[6])
print('\n----\n')

# Each entry for this item is associated with a dedicated key. For example the Stage Coordinates
print('Stage coordinates of item number 4: \n')
print(allitems[3]['StageXYZ'])
print('\n----\n')





#### Now, let's find out how to order a navigator list using the function `ordernav`.




In [ ]:
newnav = em.ordernav(allitems)

print('Let\'s look at the first points in the list...')
print(newnav[1]['# Item'])
print(newnav[2]['# Item'])
print(newnav[3]['# Item'])
print(newnav[4]['# Item'])
print(newnav[5]['# Item'])
print(newnav[6]['# Item'])
print(newnav[7]['# Item'])

# and now we need to write the ordered navigator file.

# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_sorted.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n" % navlines[1])

# fill the new file   
for nitem in newnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and this is what we get...

![The sorted navigator](images/nav1_sorted.png)


----

#### Batch process parameters of navigator items

Let's now also try and see if we can change some of the general parameters of these navigator items, like for example z-height, color or the 'Acquire' flag.

In [ ]:
# create new file by copying the header of the input file
newnavf = navfile[:-4] + '_playground.nav'
nnf = open(newnavf,'w')
nnf.write("%s\n" % navlines[0])
nnf.write("%s\n"% navlines[1])

# create a new list to collect ouput navigator items.
outnav=list()

# add the map. This we don't want to modify
outnav.append(newnav[0])

# Then modify and add the rest...

for item in newnav[1:] :
    item['StageXYZ'][2] = '31.415'
    item['Color'] = ['3']
    item['Acquire'] = ['1']
    outnav.append(item)

# fill the new file   
for nitem in outnav: 
    out = em.itemtonav(nitem,nitem['# Item'])
    for item in out: nnf.write("%s\n" % item)
            
nnf.close()

# done

----

and we get...

![The modified navigator](images/nav1_playground.png)


yellow points at the new height that are set to be acquired.

----





## Tutorial 2: Import pixel coordinates<a name="tutorial1"></a>

This tutorial will give an idea on how to import pixel cooedinates to a map.

In SerialEM, open a new empty Navigator by choosing `Navigator > Open` in the menu.

Then we import a map, this can be any kind of image, maybe from a light microscope if you're doing CLEM. Let's use the SerialEM logo from [this directory](https://git.embl.de/schorb/serialem_tools/tree/master/tutorials/Tutorial2-coordinates). Therefore, click `Navigator > Import Map` and choose a target registration, this is simply the index of the coordinate frame.

![Import Map](images/import_map.png)
![Import Map1](images/import_map2.png)

The pixel coordinates of the holes on the top left are stored in [this file](https://git.embl.de/schorb/serialem_tools/tree/master/tutorials/Tutorial1-sorting/holes.txt). Namely:

| x   | y  |
| --- |---:|
| 13|13|
|32|13|
|51|13|
|13|31|
|31|31|
|13|51|


We now would like to populate our Navigator with those.

In order to tell emTools which map to process, we mark it for acquisition.

![Acquire Map](images/acquire.png)

Then save the navigator as `nav.nav`.

Now, let's import those into our navigator.



In [11]:
# import the emTools module and make its functions available
import emtools as em

# load the navigator file
navfile = 'tutorials/Tutorial2-coordinates/nav.nav'
navlines = em.loadtext(navfile)

allitems = em.fullnav(navlines)

# find the navigator items with active 'Acquire' flag
# first find items that have an 'Acquire' entry
acq = filter(lambda item:item.get('Acquire'),allitems)
acq = list(filter(lambda item:item['Acquire']==['1'],acq))

# in this case, we have only one item
mapitem = acq[0]

# find the Registration of the map
regis = mapitem['Regis']

print('The target registration is '+ regis[0])

# import the coordinates
importlines =  em.loadtext('tutorials/Tutorial2-coordinates/holes.txt')

# create a navigator from these

newitems=list()

for idx,line in enumerate(importlines):
    
    #fill the relevant fields of the navigator item to define the point
    point = em.pointitem('Point '+str(idx))






The target registration is 2


AttributeError: 'module' object has no attribute 'pointitem'

-----



## Setting up KNIME<a name="knime"></a>

Install KNIME from [here](https://www.knime.com/downloads/download-knime)

#### enable commity nodes

Open KNIME Preferences. ("File > Preferences")
![KNIME preferences](images/knime_pref.png)


In "Install/Update > Available Software Sites", select "Stable Community Contributions" and check that "Community Contributions" is also active. 

![KNIME updates](images/knime_updates.png)


#### setting up the Python integration

In the Preferences, go to "KNIME > Python". Provide the location of the python binary executable files of your Anaconda installation. Default location in Windows would be for example: `C:\Anaconda2\python.exe`. Do this for your Python version of choice and make sure you select this version as "Default". A successful integration should show no red warning messages but simply read the version number.

![KNIME Python](images/knime_python.png)


Import the desired workflow from [here](https://git.embl.de/schorb/serialem_tools/tree/master/knime). KNIME will now ask you if it should fetch the necessary nodes from its repositories.

